<a href="https://colab.research.google.com/github/itsZENR/TicTacToe-RL/blob/two_model/TicTacToe_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Библиотеки
import numpy as np
import random
import tensorflow as tf

In [ ]:
class TicTacToeEnvironment:
    def __init__(self):
        self.state = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        self.done = False

    def get_state(self):
        return self.state.copy()

    def get_valid_actions(self):
        return np.where(self.state.flatten() == 0)[0]

    def step(self, action, current_player):
        self.current_player = current_player
        if self.done:
            raise ValueError("Game is already finished")

        i, j = np.unravel_index(action, (3, 3))
        if self.state[i, j] != 0:
            raise ValueError("Invalid action")

        self.state[i, j] = self.current_player
        winner = self.get_winner()
        if winner is not None or np.all(self.state != 0):
            self.done = True

        reward = 0
        if winner is not None:
            reward = 1 if winner == 1 else -1

        return self.get_state(), reward, self.done

    def step_game_player(self, action):
        if self.done:
              raise ValueError("Game is already finished")

        i, j = np.unravel_index(action, (3, 3))
        if self.state[i, j] != 0:
            raise ValueError("Invalid action")

        self.state[i, j] = self.current_player
        winner = self.get_winner()
        if winner is not None or np.all(self.state != 0):
            self.done = True

        reward = 0
        if winner is not None:
            reward = 1 if winner == 1 else -1

        return self.get_state(), reward, self.done

    def reset(self):
        self.state = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        self.done = False

    def get_winner(self):

        # Check rows
        for i in range(3):
            if np.all(self.state[i, :] == 1):
                return 1
            elif np.all(self.state[i, :] == -1):
                return -1

        # Check columns
        for j in range(3):
            if np.all(self.state[:, j] == 1):
                return 1
            elif np.all(self.state[:, j] == -1):
                return -1

        # Check diagonals
        if np.all(np.diag(self.state) == 1) or np.all(np.diag(np.fliplr(self.state)) == 1):
            return 1
        elif np.all(np.diag(self.state) == -1) or np.all(np.diag(np.fliplr(self.state)) == -1):
            return -1

        # Check tie
        if np.all(self.state != 0):
            return 0

        return None


In [ ]:
# Определение модели Q-функции
model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(125, activation='relu', input_shape=(9,)),
    tf.keras.layers.Dense(9)
])

model_1.compile(optimizer='Adam', loss=tf.keras.losses.BinaryCrossentropy())

# Определение модели Q-функции
model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(125, activation='relu', input_shape=(9,)),
    tf.keras.layers.Dense(9)
])

model_2.compile(optimizer='Adam', loss=tf.keras.losses.BinaryCrossentropy())

In [ ]:
# Определение функции выбора действия
def choose_action(state, epsilon, model):
    if np.random.random() < epsilon:
        # Случайное действие
        action = np.random.choice(np.where(env.state.flatten() == 0)[0])
    else:
        # Выбор действия с наибольшим значением Q-функции
        q_values = model.predict(state.reshape(1, -1))
        action = np.argmax(q_values[0] * (env.state.flatten() == 0))
        print('q_values', q_values)
    return action

# Определение функции обновления Q-функции
def update_q_function(state, action, reward, next_state, alpha, gamma, model):
    q_values = model.predict(state.reshape(1, -1), verbose=0)
    next_q_values = model.predict(next_state.reshape(1, -1), verbose=0)
    if model == model_2:
      reward = -reward
    td_target = reward + gamma * np.amax(next_q_values)
    td_error = td_target - q_values[0][action]
    q_values[0][action] += alpha * td_error
    model.fit(state.reshape(1, -1), q_values)

# Основной цикл обучения
num_episodes = 100
epsilon = 1.0
epsilon_decay = 0.9995
alpha = 0.01
gamma = 0.99

for i in range(num_episodes):
    print(i, '-'*100)
    env = TicTacToeEnvironment()
    done = False
    while not done:
        action = choose_action(env.state, epsilon, model_1)
        update_q_function(env.state, action, reward, next_state, alpha, gamma, model_1)
        next_state, reward, done = env.step(action, current_player=1)
        print("action", action)
        print(next_state)
        print('reward:', reward)
        print('done:', done)
        env.state = next_state

        if not done:
          action = choose_action(env.state, epsilon, model_2)
          update_q_function(env.state, action, reward, next_state, alpha, gamma, model_2)
          next_state, reward, done = env.step(action, current_player=-1)
          print("action", action)
          print(next_state)
          print('reward:', reward)
          print('done:', done)
          env.state = next_state


    epsilon *= epsilon_decay

Выходные данные были обрезаны до нескольких последних строк (5000).
 [ 0  1 -1]
 [ 0  1  0]]
reward: 0
done: False
1/1 [==============================] - 0s 9ms/step - loss: 0.0649
action 0
[[ 1  0 -1]
 [ 0  1 -1]
 [ 0  1  0]]
reward: 0
done: False
1/1 [==============================] - 0s 6ms/step - loss: -1.2854
action 3
[[ 1  0 -1]
 [-1  1 -1]
 [ 0  1  0]]
reward: 0
done: False
1/1 [==============================] - 0s 6ms/step - loss: 0.0788
action 8
[[ 1  0 -1]
 [-1  1 -1]
 [ 0  1  1]]
reward: 1
done: True
9 ----------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 5ms/step - loss: 0.0725
action 7
[[0 0 0]
 [0 0 0]
 [0 1 0]]
reward: 0
done: False
1/1 [==============================] - 0s 6ms/step - loss: -0.4858
action 3
[[ 0  0  0]
 [-1  0  0]
 [ 0  1  0]]
reward: 0
done: False
1/1 [==============================] - 0s 6ms/step - loss: -0.0922
action 4
[[ 0  0  0]
 [-1  1  0]
 [ 0  1  0]]
reward:

In [ ]:
model.save('model_1')

Пробуем сыграть с ботом (бот ходит первым)

In [ ]:
env = TicTacToeEnvironment()
done = False
while not done:
        action = choose_action(env.state, epsilon, model_1)
        print("action", action)
        next_state, reward, done = env.step_game_player(action)
        print(next_state)
        x, y = input("Введите координаты через пробел: ").split()
        x = int(x)
        y = int(y)
        next_state[x][y] = -1
        update_q_function(env.state, action, reward, next_state, alpha, gamma, model_1)
        print('win', env.get_winner() )
        if env.get_winner() == None:
          update_q_function(env.state, action, reward, next_state, alpha, gamma, model_1)
        else:
          done = True

action 2
[[0 0 1]
 [0 0 0]
 [0 0 0]]
Введите координаты через пробел: 1 1
[[ 0  0  1]
 [ 0 -1  0]
 [ 0  0  0]]
win None
1/1 [==============================] - 0s 11ms/step - loss: 0.1705
action 4
[[0 0 1]
 [0 1 0]
 [0 0 0]]


KeyboardInterrupt: ignored